In [1]:
# write in train.py
import sys, os
sys.path.append(os.pardir)

In [2]:
import tensorflow as tf
import json
import argparse

from data_utils import Data
from models.char_cnn_zhang import CharCNNZhang
from models.char_cnn_kim import CharCNNKim

Using TensorFlow backend.


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, default='char_cnn_zhang', help='Specifies which model to use: char_cnn_zhang or char_cnn_kim')
FLAGS = parser.parse_args(["--model", "char_cnn_zhang"])

# Load configurations
config = json.load(open('../config.json'))

# change key from 'model' to 'char_cnn_zhang'
model_name = config['model'] # char_cnn_zhang
config['model'] = config[model_name]

# Set the data path in order to run in the notebook 
config['data']["training_data_source"] = '../data/ag_news_csv/train.csv'
config['data']["validation_data_source"] = '../data/ag_news_csv/test.csv'

# Load training data
training_data = Data(data_source=config["data"]["training_data_source"],
                     alphabet=config["data"]["alphabet"],
                     input_size=config["data"]["input_size"],
                     num_of_classes=config["data"]["num_of_classes"])
training_data.load_data()
training_inputs, training_labels = training_data.get_all_data()

# Load validation data
validation_data = Data(data_source=config["data"]["validation_data_source"],
                       alphabet=config["data"]["alphabet"],
                       input_size=config["data"]["input_size"],
                       num_of_classes=config["data"]["num_of_classes"])
validation_data.load_data()
validation_inputs, validation_labels = validation_data.get_all_data()


Data loaded from ../data/ag_news_csv/train.csv
Data loaded from ../data/ag_news_csv/test.csv


In [6]:
config

{'char_cnn_kim': {'conv_layers': [[256, 10], [256, 7], [256, 5], [256, 3]],
  'dropout_p': 0.1,
  'embedding_size': 128,
  'fully_connected_layers': [1024, 1024],
  'loss': 'categorical_crossentropy',
  'optimizer': 'adam',
  'threshold': 1e-06},
 'char_cnn_zhang': {'conv_layers': [[256, 7, 3],
   [256, 7, 3],
   [256, 3, -1],
   [256, 3, -1],
   [256, 3, -1],
   [256, 3, 3]],
  'dropout_p': 0.5,
  'embedding_size': 128,
  'fully_connected_layers': [1024, 1024],
  'loss': 'categorical_crossentropy',
  'optimizer': 'adam',
  'threshold': 1e-06},
 'data': {'alphabet': 'abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:\'"/\\|_@#$%^&*~`+-=<>()[]{}',
  'alphabet_size': 69,
  'input_size': 1014,
  'num_of_classes': 4,
  'training_data_source': '../data/ag_news_csv/train.csv',
  'validation_data_source': '../data/ag_news_csv/test.csv'},
 'model': {'conv_layers': [[256, 7, 3],
   [256, 7, 3],
   [256, 3, -1],
   [256, 3, -1],
   [256, 3, -1],
   [256, 3, 3]],
  'dropout_p': 0.5,
  'embedding_size': 

In [5]:
config['model']

{'conv_layers': [[256, 7, 3],
  [256, 7, 3],
  [256, 3, -1],
  [256, 3, -1],
  [256, 3, -1],
  [256, 3, 3]],
 'dropout_p': 0.5,
 'embedding_size': 128,
 'fully_connected_layers': [1024, 1024],
 'loss': 'categorical_crossentropy',
 'optimizer': 'adam',
 'threshold': 1e-06}

In [17]:
config['data']

{'alphabet': 'abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:\'"/\\|_@#$%^&*~`+-=<>()[]{}',
 'alphabet_size': 69,
 'input_size': 1014,
 'num_of_classes': 4,
 'training_data_source': '../data/ag_news_csv/train.csv',
 'validation_data_source': '../data/ag_news_csv/test.csv'}

In [4]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Dense


In [19]:
input_size = config['data']['input_size'] # config['data']['input_size'] = 1014
alphabet_size = config['data']['alphabet_size'] # 69
embedding_size = config['model']['embedding_size'] # 128

In [20]:
# Input 
inputs = Input(shape=(input_size,), name='sent_input', dtype='int64')  # shape=(?, 1014)
embeddings = Embedding(alphabet_size, embedding_size)